In [1]:
#
# Copyright © 2019 Sunho Kim. All rights reserved.
#

In [2]:
cd ..

/home/sunho/Documents/dev/gorani-reader/backend/dataserver


In [3]:
# parameters
lib_path = './in'

In [4]:
from os import listdir
from os.path import isfile, join
files = [join(lib_path, f) for f in listdir(lib_path) if isfile(join(lib_path, f)) and f.endswith('.epub')]
print(files)

['./in/It-A-Novel.epub']


In [6]:
import pyspark.sql.functions as F
from gorani.spark import write_data, read_data_all
from ebooklib import epub
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def get_id(file):
    import re
    p = re.compile('.+?([0-9]+?)\.epub')
    match = p.match(file)
    return int(match.group(1))

def clean_html(html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html)
    for script in soup(['script', 'style']):
        script.extract()

    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

def convert_to_book_words(self, df, id):
    df = df.groupBy('word').count()\
        .select('word', F.col('count').alias('n'))\
        .withColumn('book_id', F.lit(id))
    write_data('book_words',df)

def covert_to_book(df, title, id):
    df = df.agg(F.collect_list('word').alias('content'))\
    .withColumn('id', F.lit(id))\
    .withColumn('name', F.lit(title))
    write_data('books',df)

for file in files:
    lemmatizer = WordNetLemmatizer()
    engstopwords = set(stopwords.words('english'))

    book = epub.read_epub(file)
    # get html text from epub > words array
    df = sc.parallelize(book.items)\
    .filter(lambda item: isinstance(item, epub.EpubHtml))\
    .map(lambda item: item.get_content())\
    .map(clean_html) \
    .flatMap(word_tokenize)\
    .map(lambda s: lemmatizer.lemmatize(s))\
    .map(lambda s: s.lower())\
    .filter(lambda s: s not in engstopwords)\
    .toDF('string')

    # filter out the words not in dictionary
    words = read_data_all(spark, 'words', cache=True)
    sig_df = df.join(words, df['value'] == words['word'], 'inner')\
    .select('word').cache()

    convert_to_book_words(sig_df, id)
    covert_to_book(sig_df, book.title, id)





Py4JJavaError: An error occurred while calling o107.load.
: java.io.IOException: Failed to open native connection to Cassandra at {127.0.0.1}:9042
	at com.datastax.spark.connector.cql.CassandraConnector$.com$datastax$spark$connector$cql$CassandraConnector$$createSession(CassandraConnector.scala:168)
	at com.datastax.spark.connector.cql.CassandraConnector$$anonfun$8.apply(CassandraConnector.scala:154)
	at com.datastax.spark.connector.cql.CassandraConnector$$anonfun$8.apply(CassandraConnector.scala:154)
	at com.datastax.spark.connector.cql.RefCountedCache.createNewValueAndKeys(RefCountedCache.scala:32)
	at com.datastax.spark.connector.cql.RefCountedCache.syncAcquire(RefCountedCache.scala:69)
	at com.datastax.spark.connector.cql.RefCountedCache.acquire(RefCountedCache.scala:57)
	at com.datastax.spark.connector.cql.CassandraConnector.openSession(CassandraConnector.scala:79)
	at com.datastax.spark.connector.cql.CassandraConnector.withSessionDo(CassandraConnector.scala:111)
	at com.datastax.